In [3]:
cd /workspace/CMBA/HRM

/workspace/CMBA/HRM


In [4]:
!python3 -m pip install -U datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 92.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 161.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 295.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 282.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [datasets]/16 [datasets]ess]


In [8]:
!python dataset/build_gsm8k_bpe.py \
  --output-dir data/gsm8k-bpe-2048 \
  --max-seq-len 2048 \
  --vocab-size 32000

README.md: 7.94kB [00:00, 38.6MB/s]
main/train-00000-of-00001.parquet: 100%|███| 2.31M/2.31M [00:00<00:00, 7.02MB/s]
main/test-00000-of-00001.parquet: 100%|██████| 419k/419k [00:00<00:00, 3.75MB/s]
Generating test split: 100%|█████| 1319/1319 [00:00<00:00, 552289.80 examples/s]
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/gsm8k_sp.txt
  input_format: 
  model_prefix: data/gsm8k_sp
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  tr

In [25]:
!python dataset/build_gsm8k_bpe.py \
  --output-dir data/gsm8k-bpe-1024 \
  --max-seq-len 1024 \
  --vocab-size 32000

100%|█████████████████████████████████████| 1319/1319 [00:00<00:00, 5034.01it/s]


In [27]:
!DISABLE_COMPILE=1 PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True GRAD_CLIP_NORM=1.0 \
python pretrain.py \
  arch=hrm_4l_v4 data_path=data/gsm8k-bpe-1024 \
  epochs=10 eval_interval=null checkpoint_every_eval=true \
  global_batch_size=2 lr=1e-4 lr_min_ratio=0.1 lr_warmup_steps=2000 \
  puzzle_emb_lr=1e-4 weight_decay=0.1 puzzle_emb_weight_decay=0.0 \
  arch.hidden_size=512 arch.num_heads=8 \
  arch.v_head_dim=64 arch.qk_nope_head_dim=64 arch.qk_rope_head_dim=32 arch.kv_lora_rank=256 \
  arch.C_layers=2 arch.M_layers=2 arch.B_layers=2 arch.A_layers=2 \
  arch.halt_max_steps=16 arch.A_cycles=2 arch.M_per_B=2 arch.B_per_A=1 \
  arch.causal_in_a=true arch.attn_impl=naive

  0%|                                                 | 0/37365 [00:00<?, ?it/s]wandb: Currently logged in as: viharikvs (viharikvs-urbankisaan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: ⣽ Waiting for wandb.init()...
wandb: ⣾ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.21.2
wandb: Run data is saved locally in /workspace/CMBA/HRM/wandb/run-20250829_114547-gv95csh1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run HierarchicalReasoningModel4L_ACTV3 wild-skink
wandb: ⭐️ View project at https://wandb.ai/viharikvs-urbankisaan/Gsm8k-bpe-1024%20ACT-torch
wandb: 🚀 View run at https://wandb.ai/vi

In [29]:
!python dataset/build_gsm8k_bpe.py \
  --output-dir data/gsm8k-bpe-1024-digits \
  --max-seq-len 1024 \
  --vocab-size 32000 \
  --labels-mode digits

100%|█████████████████████████████████████| 1319/1319 [00:00<00:00, 4216.47it/s]


In [36]:
!python dataset/build_gsm8k_bpe.py \
  --output-dir data/gsm8k-bpe-1024-digits-final \
  --max-seq-len 1024 --vocab-size 32000 \
  --labels-mode digits --no-include-rationale

100%|█████████████████████████████████████| 1319/1319 [00:00<00:00, 9995.80it/s]


In [37]:
ls -1 data/gsm8k-bpe-1024-digits-final/train/dataset.json

data/gsm8k-bpe-1024-digits-final/train/dataset.json


In [39]:
!DISABLE_COMPILE=1 PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True GRAD_CLIP_NORM=1.0 \
python pretrain.py \
  arch=hrm_4l_v4 data_path=data/gsm8k-bpe-1024-digits-final \
  epochs=10 eval_interval=null checkpoint_every_eval=true \
  global_batch_size=4 lr=1e-4 lr_min_ratio=0.1 lr_warmup_steps=2000 \
  puzzle_emb_lr=1e-4 weight_decay=0.1 puzzle_emb_weight_decay=0.0 \
  arch.hidden_size=512 arch.num_heads=8 \
  arch.v_head_dim=64 arch.qk_nope_head_dim=64 arch.qk_rope_head_dim=32 arch.kv_lora_rank=256 \
  arch.C_layers=2 arch.M_layers=2 arch.B_layers=2 arch.A_layers=2 \
  arch.halt_max_steps=1 arch.A_cycles=1 arch.M_per_B=1 arch.B_per_A=1 \
  arch.causal_in_a=true arch.attn_impl=naive arch.output_vocab_size=16 \
  arch.aux_recon_loss_weight=0.5

  0%|                                                 | 0/18682 [00:00<?, ?it/s]wandb: Currently logged in as: viharikvs (viharikvs-urbankisaan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.21.2
wandb: Run data is saved locally in /workspace/CMBA/HRM/wandb/run-20250829_121257-0cekkbdb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run HierarchicalReasoningModel4L_ACTV3 invaluable-zebra
wandb: ⭐️ View project at https://wandb.ai/viharikvs-urbankisaan/Gsm8k-bpe-1024-digits-final%20ACT-torch
wandb: 🚀 View run at https://wandb.ai/viharikvs-urbankisaan/Gsm8k-bpe-1024-digits-final%20ACT-t

In [40]:
!find checkpoints -type f -name 'step_*' | cat

checkpoints/Gsm8k-bpe-1024-digits-final ACT-torch/HierarchicalReasoningModel4L_ACTV3 invaluable-zebra/step_18682
checkpoints/Maze-30x30-hard-1k ACT-torch/HierarchicalReasoningModel_ACTV1 spotted-markhor/step_781_all_preds.0
checkpoints/Maze-30x30-hard-1k ACT-torch/HierarchicalReasoningModel_ACTV1 spotted-markhor/step_781
checkpoints/Sudoku-quick-test ACT-torch/HierarchicalReasoningModel4L_ACTV3 proficient-shrimp/step_3200


In [44]:
!DISABLE_COMPILE=1 python evaluate.py checkpoint="checkpoints/Gsm8k-bpe-1024-digits-final ACT-torch/HierarchicalReasoningModel4L_ACTV3 invaluable-zebra/step_18682"

Starting evaluation
{'all': {'accuracy': 0.4146033525466919, 'exact_accuracy': 0.018953753635287285, 'lm_loss': 2.315401077270508, 'q_halt_accuracy': 0.9810462594032288, 'q_halt_loss': 0.19021552801132202, 'steps': 1.0}}


In [ ]:
wandb: 
wandb: Run history:
wandb:            num_params ▁
wandb:        train/accuracy ▂▄▆▄▃▃▅▂▄▃▄▆▁▄▂▆▆▄▃▇▂▂▆▄▃▃█▅▂▅▇▃▆▃▅▅▃▁▃▆
wandb:           train/count ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:  train/exact_accuracy ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁
wandb:         train/lm_loss ▂▁▂▅▂▁▂▅▂▁▃▁▁▄▂▂▁▂▄▆▂▂▃▁▂▁▂▂▄▁▁▂▁▁▇▇▂▂▁█
wandb:              train/lr ▁▂▂▄▄▅█████▇▇▇▇▇▇▆▆▆▆▆▅▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
wandb: train/q_halt_accuracy ███████████▁████████████▁██▁██████████▁█
wandb:     train/q_halt_loss ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁█▁▁▁▁▁▁▁▁
wandb:           train/steps ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:            num_params 213310979
wandb:        train/accuracy 0.52083
wandb:           train/count 1
wandb:  train/exact_accuracy 0
wandb:         train/lm_loss 1.39293
wandb:              train/lr 1e-05
wandb: train/q_halt_accuracy 1
wandb:     train/q_halt_loss 3e-05
wandb:           train/steps 1
wandb: 
wandb: 🚀 View run HierarchicalReasoningModel4L_ACTV3 invaluable-zebra at: https://wandb.ai/viharikvs-urbankisaan/Gsm8k-bpe-1024-digits-final%20ACT-torch/runs/0cekkbdb
wandb: ⭐️ View project at: https://wandb.ai/viharikvs-urbankisaan/Gsm8k-bpe-1024-digits-final%20ACT-torch
wandb: Synced 5 W&B file(s), 0 media file(s), 3 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250829_121257-0cekkbdb/logs
100%|█| 18682/18682 [43:55<00:00,  7.09it/s, train/accuracy=0.479, train/count=1